# View Job Request

View the `submission_request` stored for a job to verify API request logging.

**Purpose:**
- Inspect the actual HTTP request body sent to Moody's API
- Debug job submission issues
- Verify request payload logging is working correctly

In [ ]:
import sys
import json
from pathlib import Path

workspace = Path.cwd().parent.parent
if str(workspace) not in sys.path:
    sys.path.insert(0, str(workspace))

from helpers.database import execute_query
from helpers import ux

## 1) Enter Job ID

In [ ]:
# Enter the job ID to inspect
job_id = 2900  # <-- Change this

## 2) View Job Details

In [ ]:
result = execute_query("""
    SELECT 
        j.id,
        j.status,
        j.moodys_workflow_id,
        j.submission_request,
        j.submission_response,
        j.created_ts,
        j.submitted_ts,
        b.batch_type
    FROM irp_job j
    JOIN irp_batch b ON j.batch_id = b.id
    WHERE j.id = %s
""", (job_id,))

if result.empty:
    ux.warning(f"No job found with ID {job_id}")
else:
    row = result.iloc[0]
    ux.subheader("Job Info")
    ux.table([
        ["Job ID", row['id']],
        ["Status", row['status']],
        ["Batch Type", row['batch_type']],
        ["Moody's Workflow ID", row['moodys_workflow_id']],
        ["Created", row['created_ts']],
        ["Submitted", row['submitted_ts']]
    ], headers=["Field", "Value"])

## 3) View Submission Request

In [ ]:
if not result.empty:
    data = row['submission_request']
    
    if data is None:
        ux.warning("submission_request is NULL (not populated)")
    else:
        ux.subheader("Full Submission Request")
        print(json.dumps(data, indent=2, default=str))
        
        # Highlight http_request_body if present
        if isinstance(data, dict) and 'http_request_body' in data:
            ux.subheader("HTTP Request Body (actual API payload)")
            print(json.dumps(data['http_request_body'], indent=2, default=str))

## 4) View Submission Response

In [ ]:
if not result.empty:
    response = row['submission_response']
    
    if response is None:
        ux.warning("submission_response is NULL")
    else:
        ux.subheader("Submission Response")
        print(json.dumps(response, indent=2, default=str))